In [0]:
from pyspark.sql.functions import col, current_timestamp
spark.conf.set(
  "fs.azure.account.key.snowflakehexa.blob.core.windows.net",
  "s+dpBFiy973QoHY+hvsWQHQOtSglbQMOLcSPJzr6TT35DnKIzpdHgDuXHg7Ss9maUparnjBtbxrt+AStvpvY3A=="
)


In [0]:
storage_account = "snowflakehexa"
container = "raw-sales-data"
file_name = "sales_transactions_2024_01_01.csv"

file_path = f"wasbs://raw-sales-data@snowflakehexa.blob.core.windows.net/sales_transactions_2024_01_01.csv"


In [0]:
df_raw = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(file_path)

df_raw.show()


+--------+-----------+------+----------------+--------+----------+----------+
|order_id|customer_id|region|product_category|quantity|unit_price|order_date|
+--------+-----------+------+----------------+--------+----------+----------+
|    1001|    CUST101|  APAC|     Electronics|       2|     45000|2024-01-01|
|    1002|    CUST102|    US|       Furniture|       1|     78000|2024-01-01|
|    1003|    CUST103|    EU|        Clothing|       5|      3200|2024-01-02|
|    1004|    CUST104|  APAC|     Electronics|       1|     55000|2024-01-02|
|    1005|    CUST105|    US|       Groceries|      10|       450|2024-01-03|
+--------+-----------+------+----------------+--------+----------+----------+



In [0]:
df_valid = df_raw.filter(
    col("order_id").isNotNull() &
    (col("quantity") > 0) &
    (col("unit_price") > 0)
)

df_valid.show()


+--------+-----------+------+----------------+--------+----------+----------+
|order_id|customer_id|region|product_category|quantity|unit_price|order_date|
+--------+-----------+------+----------------+--------+----------+----------+
|    1001|    CUST101|  APAC|     Electronics|       2|     45000|2024-01-01|
|    1002|    CUST102|    US|       Furniture|       1|     78000|2024-01-01|
|    1003|    CUST103|    EU|        Clothing|       5|      3200|2024-01-02|
|    1004|    CUST104|  APAC|     Electronics|       1|     55000|2024-01-02|
|    1005|    CUST105|    US|       Groceries|      10|       450|2024-01-03|
+--------+-----------+------+----------------+--------+----------+----------+



In [0]:
df_transformed = df_valid.withColumn(
    "total_amount", col("quantity") * col("unit_price")
).withColumn(
    "load_timestamp", current_timestamp()
)

df_transformed.show()


+--------+-----------+------+----------------+--------+----------+----------+------------+--------------------+
|order_id|customer_id|region|product_category|quantity|unit_price|order_date|total_amount|      load_timestamp|
+--------+-----------+------+----------------+--------+----------+----------+------------+--------------------+
|    1001|    CUST101|  APAC|     Electronics|       2|     45000|2024-01-01|       90000|2025-12-22 13:32:...|
|    1002|    CUST102|    US|       Furniture|       1|     78000|2024-01-01|       78000|2025-12-22 13:32:...|
|    1003|    CUST103|    EU|        Clothing|       5|      3200|2024-01-02|       16000|2025-12-22 13:32:...|
|    1004|    CUST104|  APAC|     Electronics|       1|     55000|2024-01-02|       55000|2025-12-22 13:32:...|
|    1005|    CUST105|    US|       Groceries|      10|       450|2024-01-03|        4500|2025-12-22 13:32:...|
+--------+-----------+------+----------------+--------+----------+----------+------------+--------------

In [0]:
sfOptions = {
  "sfURL": "GFCCSWR-TW94786.snowflakecomputing.com",
  "sfDatabase": "RETAIL_DB",
  "sfSchema": "SALES",
  "sfWarehouse": "RETAIL_WH",
  "sfRole": "SYSADMIN",
  "sfUser": "RAMKISHORESN",
  "sfPassword": "Ramkishoresn@06"
}


In [0]:
from pyspark.sql.types import IntegerType, DoubleType, DateType

df_final = df_transformed.select(
    col("order_id").cast(IntegerType()),
    col("customer_id"),
    col("region"),
    col("product_category"),
    col("quantity").cast(IntegerType()),
    col("unit_price").cast(DoubleType()),
    col("total_amount").cast(DoubleType()),
    col("order_date").cast(DateType()),
    col("load_timestamp")
)


In [0]:
df_final.write \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "SALES_TRANSACTIONS") \
    .mode("append") \
    .save()